In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 

dataset_path = os .listdir('dataset/train')
label_types = os.listdir('dataset/train')
print(label_types)

['v_CricketShot_g08_c01.avi', 'v_CricketShot_g08_c02.avi', 'v_CricketShot_g08_c03.avi', 'v_CricketShot_g08_c04.avi', 'v_CricketShot_g08_c05.avi', 'v_CricketShot_g08_c06.avi', 'v_CricketShot_g08_c07.avi', 'v_CricketShot_g09_c01.avi', 'v_CricketShot_g09_c02.avi', 'v_CricketShot_g09_c03.avi', 'v_CricketShot_g09_c04.avi', 'v_CricketShot_g09_c05.avi', 'v_CricketShot_g09_c06.avi', 'v_CricketShot_g09_c07.avi', 'v_CricketShot_g10_c01.avi', 'v_CricketShot_g10_c02.avi', 'v_CricketShot_g10_c03.avi', 'v_CricketShot_g10_c04.avi', 'v_CricketShot_g10_c05.avi', 'v_CricketShot_g10_c06.avi', 'v_CricketShot_g10_c07.avi', 'v_CricketShot_g11_c01.avi', 'v_CricketShot_g11_c02.avi', 'v_CricketShot_g11_c03.avi', 'v_CricketShot_g11_c04.avi', 'v_CricketShot_g11_c05.avi', 'v_CricketShot_g11_c06.avi', 'v_CricketShot_g11_c07.avi', 'v_CricketShot_g12_c01.avi', 'v_CricketShot_g12_c02.avi', 'v_CricketShot_g12_c03.avi', 'v_CricketShot_g12_c04.avi', 'v_CricketShot_g12_c05.avi', 'v_CricketShot_g12_c06.avi', 'v_CricketSho

In [ ]:
!pip install git+https://github.com/tensorflow/docs
    

In [ ]:
pip install imutils

In [2]:
from tensorflow_docs.vis import embed
from tensorflow import keras 
from imutils import paths



import matplotlib.pyplot as plt 
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import imageio
import os

In [3]:
gpus=tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RuntimeError as e:
        print(e)

In [4]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

print(f"Total Videos for training: {len(train_df)}")
print(f"Total Videos for testing: {len(test_df)}")

train_df.sample(10)

Total Videos for training: 594
Total Videos for training: 224


,video_name,tag
313,v_Punch_g19_c01.avi,Punch
445,v_ShavingBeard_g21_c03.avi,ShavingBeard
409,v_ShavingBeard_g16_c01.avi,ShavingBeard
554,v_TennisSwing_g19_c03.avi,TennisSwing
59,v_CricketShot_g16_c04.avi,CricketShot
102,v_CricketShot_g23_c04.avi,CricketShot
344,v_Punch_g23_c07.avi,Punch
168,v_PlayingCello_g15_c06.avi,PlayingCello
395,v_ShavingBeard_g13_c06.avi,ShavingBeard
249,v_Punch_g09_c05.avi,Punch


In [5]:
# Feeding our input or the videos to the network 
IMG_SIZE = 224


def crop_centre_square(frame):
    y,x = frame.shape[0:2]
    min_dim = min(y,x)
    start_x = (x//2) - (min_dim//2)
    start_y = (y//2) - (min_dim//2)
    return frame[start_y : start_y + min_dim , start_x + min_dim]

def load_video(path,max_frames=0,resize=(IMG_SIZE,IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames=[]
    try:
        while True:
            ret,frame = cap.read()
            if not ret:
                break
            frame = crop_centre_square(frame)
            frame = cv2.resize(frame,resize)
            frame = frame[:,:,[2,1,0]]
            frames.append(frame)
            
            if len(frames)==max_frames:
                break
                
    finally :
        cap.release()
        return np.array(frames)

In [6]:
import keras
from keras.applications import InceptionV3

def build_feature_extractor():
    feature_extractor = InceptionV3(
        weights="imagenet",  # Corrected typo here
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    
    preprocess_input = keras.applications.inception_v3.preprocess_input
    
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)
    
    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

IMG_SIZE = 299  # Example size, adjust according to your needs
feature_extractor = build_feature_extractor()


In [7]:
# Label encoding 

label_processor = keras.layers.StringLookup(num_oov_indices=0,vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels


['CricketShot', 'PlayingCello', 'Punch', 'ShavingBeard', 'TennisSwing']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [8]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS =100

MAX_SEQ_LENGTH =20
NUM_FEATURES = 2048

import cv2
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

In [10]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(10)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(10, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

def run_experiment():
    filepath = "/tmp/video_classifier.weights.h5"  # Adjusted filepath
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model
_, sequence_model = run_experiment()

Epoch 1/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2933 - loss: 1.6084
Epoch 1: val_loss improved from inf to 1.62252, saving model to /tmp/video_classifier.weights.h5
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 108ms/step - accuracy: 0.2888 - loss: 1.6081 - val_accuracy: 0.0000e+00 - val_loss: 1.6225
Epoch 2/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2758 - loss: 1.6020
Epoch 2: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.2778 - loss: 1.6019 - val_accuracy: 0.0000e+00 - val_loss: 1.6362
Epoch 3/100
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3071 - loss: 1.5961
Epoch 3: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3038 - loss: 1.5960 - val_accuracy: 0.0000e+00 - val_loss: 1.6493
Epoch 4/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2923 - loss: 1.5906
Epoch 4: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.290

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3004 - loss: 1.5006
Epoch 26: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2993 - loss: 1.5009 - val_accuracy: 0.0000e+00 - val_loss: 1.9340
Epoch 27/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3027 - loss: 1.4990
Epoch 27: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3014 - loss: 1.4992 - val_accuracy: 0.0000e+00 - val_loss: 1.9451
Epoch 28/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3120 - loss: 1.5012
Epoch 28: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3071 - loss: 1.5006 - val_accuracy: 0.0000e+00 - val_loss: 1.9564
Epoch 29/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2628 - loss: 1.4878
Epoch 29: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.2644 - loss: 1.4884 - val_accuracy: 0.0000e+00 - val_loss: 

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2728 - loss: 1.4492 - val_accuracy: 0.0000e+00 - val_loss: 2.1872
Epoch 52/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3176 - loss: 1.4434
Epoch 52: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3128 - loss: 1.4455 - val_accuracy: 0.0000e+00 - val_loss: 2.1964
Epoch 53/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2973 - loss: 1.4467
Epoch 53: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2955 - loss: 1.4479 - val_accuracy: 0.0000e+00 - val_loss: 2.2051
Epoch 54/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3184 - loss: 1.4356
Epoch 54: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3165 - loss: 1.4366 - val_accuracy: 0.0000e+00 - val_loss: 2.2144
Epoch 55/100
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2840 - loss: 1.4606
Epoch 55: val_loss did not i

Epoch 77/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2934 - loss: 1.4442
Epoch 77: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2928 - loss: 1.4402 - val_accuracy: 0.0000e+00 - val_loss: 2.3983
Epoch 78/100
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3298 - loss: 1.4351
Epoch 78: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3244 - loss: 1.4335 - val_accuracy: 0.0000e+00 - val_loss: 2.4056
Epoch 79/100
11/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3078 - loss: 1.4343
Epoch 79: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3049 - loss: 1.4319 - val_accuracy: 0.0000e+00 - val_loss: 2.4131
Epoch 80/100
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3098 - loss: 1.4179
Epoch 80: val_loss did not improve from 1.62252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3072 - loss: 1.4184 - val_accuracy: 0.0000e+00

In [11]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: v_PlayingCello_g05_c07.avi
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
  CricketShot: 20.22%
  Punch: 20.21%
  PlayingCello: 20.08%
  ShavingBeard: 19.76%
  TennisSwing: 19.73%


In [1]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="dataset/test/dancing/dancing (23.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")
